In [2]:
import pandas
import numpy as np
from numpy.linalg import inv
import torch
from torch.autograd import Variable
import torch .nn. functional as F

In [3]:
#read data
data=pandas.read_csv('df_NaN.csv')

In [4]:
#set Developing to 1 and Developed to 0
data['Status'][data['Status'] == 'Developing']  = 0
data['Status'][data['Status'] == 'Developed']  = 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
#extract target data

target=np.array(data['Life expectancy '])

#Ignore the country and year 
data=data.drop(['Country','Year','Unnamed: 0','Life expectancy '],axis=1)



In [6]:
#extract normalization parameters and normalize original data
dataMax=np.array(data.max())
dataMin=np.array(data.min())
dataMean=np.array(data.mean())
data=data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
# is it necessary to normalize label data? seems not
life=target.copy()
#target=(target-target.min())/(target.max()-target.min())

In [7]:
train_x=np.array(data)

In [8]:
def least_square(train_x, train_y):
    """
    use least square method to reach the regression model
    input：
          train_x, training data
          train_y, labels in training data
    """  
    weights = inv(np.dot(train_x.T ,train_x).astype(float)).dot(train_x.T).dot(train_y) 
    return weights.astype(float)

least_square_weights=least_square(train_x,target)
#prove to be very inaccurate

In [9]:
prediction=np.dot(train_x,least_square_weights)

In [11]:
MSE=np.sum((target-prediction)**2)/prediction.size

In [12]:
print('MSE of leas square method',MSE)

78.25739275132592

In [8]:
"""
#not working, need to debug later
def gradient_descent(train_x, train_y, maxCycle, alpha):
     
    numSamples, numFeatures = np.shape(train_x)
    weights = np.zeros((numFeatures,1))
     
    for i in range(maxCycle):
        h = train_x.dot(weights)
        err = h - train_y           
        weights = weights - (alpha*err.T.dot(train_x)).T
        print(err)
    return weights.astype(float)
"""


'\ndef gradient_descent(train_x, train_y, maxCycle, alpha):\n     \n    numSamples, numFeatures = np.shape(train_x)\n    weights = np.zeros((numFeatures,1))\n     \n    for i in range(maxCycle):\n        h = train_x.dot(weights)\n        err = h - train_y           \n        weights = weights - (alpha*err.T.dot(train_x)).T\n        print(err)\n    return weights.astype(float)\n'

In [18]:
#train a shallow neural network with pytorch


#convert numpy data to torch variables
#use GPU if avaiable
if torch.cuda.is_available():
    torch_x=Variable(torch.from_numpy(train_x.astype(float)).cuda(),requires_grad = True)
    torch_y=Variable(torch.from_numpy(target).cuda(),requires_grad = False)
else:
    torch_x=Variable(torch.from_numpy(train_x.astype(float)),requires_grad = True)
    torch_y=Variable(torch.from_numpy(target),requires_grad = False)

class ShallowModel(torch.nn.Module):
    def __init__(self):
        super(ShallowModel, self).__init__()
        self.fc1 = torch.nn.Linear(19,57).double()
        #self.fc2 = torch.nn.Linear(19,19).double()
        self.fc2 = torch.nn.Linear(57,1).double()
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        #x=F.relu(self.fc2(x))
        x=self.fc2(x)
        return x
    
class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.fc1 = torch.nn.Linear(19,1).double()
        #self.fc2 = torch.nn.Linear(19,19).double()
        #self.fc2 = torch.nn.Linear(57,1).double()
        
    def forward(self,x):
        #x=F.relu(self.fc1(x))
        #x=F.relu(self.fc2(x))
        x=self.fc1(x)
        return x
    

            
         
  

In [99]:
def train_model(model,epochs,trainsize=0.8):
    assert isinstance(model,str)
    assert isinstance(epochs,int) and epochs>1
    
    if torch.cuda.is_available():
        if model=='shallow':
            model=ShallowModel().cuda()
        elif model=='linear':
            model=LinearModel().cuda()
    else:
        if model=='shallow':
            model=ShallowModel()
        elif model=='linear':
            model=LinearModel()
    
    criterion=torch.nn.MSELoss()
    optimizer= torch.optim.SGD(model.parameters(), lr=0.01)
    
    #separate trainning set and testset
    #shuffle the training set
    
    shuffleindex=np.arange(torch_x.size()[0])
    np.random.shuffle(shuffleindex)

    train_index=int(trainsize*torch_x.size()[0])
    
    train_x=torch_x[shuffleindex[0:train_index]]
    train_y=torch_y[shuffleindex[0:train_index]]
    
    test_x=torch_x[shuffleindex[train_index:]]
    test_y=torch_y[shuffleindex[train_index:]]
    
    T=epochs
    B=100
    NB=int(train_index/100)+1
    N=train_index
    #train the model
    for epoch in range(T):
        running_loss=0.0
        idxminibatches = np. random . permutation (NB)
        for k in range(NB):
            i = idxminibatches [k]
            idxsmp=np.arange(B*i,min(B*(i+1),N))
            inputs = torch_x[idxsmp]
            labels = torch_y[idxsmp]

            optimizer.zero_grad()

            y_pred=model(inputs)

            loss=criterion(y_pred,labels)

            loss.backward()

            optimizer.step()
            running_loss+=loss[0]
            if k==NB-1:
                print(epoch,running_loss/NB)
                running_loss=0.0
                
    #assess the model on test set
    
    y_pred=model(test_x)
    loss=criterion(y_pred,test_y)
    print('MSE on test set',loss[0])
    
    return model

In [78]:
train_model('linear',2000)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(1306.9562, dtype=torch.float64, device='cuda:0')
1 tensor(95.4880, dtype=torch.float64, device='cuda:0')
2 tensor(76.6649, dtype=torch.float64, device='cuda:0')
3 tensor(74.0960, dtype=torch.float64, device='cuda:0')
4 tensor(71.8235, dtype=torch.float64, device='cuda:0')
5 tensor(69.4185, dtype=torch.float64, device='cuda:0')
6 tensor(67.5537, dtype=torch.float64, device='cuda:0')
7 tensor(65.5957, dtype=torch.float64, device='cuda:0')
8 tensor(63.7138, dtype=torch.float64, device='cuda:0')
9 tensor(61.9975, dtype=torch.float64, device='cuda:0')
10 tensor(60.2510, dtype=torch.float64, device='cuda:0')
11 tensor(58.5456, dtype=torch.float64, device='cuda:0')
12 tensor(57.3993, dtype=torch.float64, device='cuda:0')
13 tensor(55.7362, dtype=torch.float64, device='cuda:0')
14 tensor(54.4365, dtype=torch.float64, device='cuda:0')
15 tensor(53.1170, dtype=torch.float64, device='cuda:0')
16 tensor(51.8859, dtype=torch.float64, device='cuda:0')
17 tensor(50.5676, dtype=torch.float64,

152 tensor(17.8514, dtype=torch.float64, device='cuda:0')
153 tensor(17.8448, dtype=torch.float64, device='cuda:0')
154 tensor(17.7826, dtype=torch.float64, device='cuda:0')
155 tensor(17.7055, dtype=torch.float64, device='cuda:0')
156 tensor(17.7013, dtype=torch.float64, device='cuda:0')
157 tensor(17.6460, dtype=torch.float64, device='cuda:0')
158 tensor(17.6021, dtype=torch.float64, device='cuda:0')
159 tensor(17.5758, dtype=torch.float64, device='cuda:0')
160 tensor(17.4997, dtype=torch.float64, device='cuda:0')
161 tensor(17.4671, dtype=torch.float64, device='cuda:0')
162 tensor(17.4155, dtype=torch.float64, device='cuda:0')
163 tensor(17.3937, dtype=torch.float64, device='cuda:0')
164 tensor(17.3067, dtype=torch.float64, device='cuda:0')
165 tensor(17.3269, dtype=torch.float64, device='cuda:0')
166 tensor(17.1816, dtype=torch.float64, device='cuda:0')
167 tensor(17.2436, dtype=torch.float64, device='cuda:0')
168 tensor(17.1829, dtype=torch.float64, device='cuda:0')
169 tensor(17.

294 tensor(14.6254, dtype=torch.float64, device='cuda:0')
295 tensor(14.5656, dtype=torch.float64, device='cuda:0')
296 tensor(14.5866, dtype=torch.float64, device='cuda:0')
297 tensor(14.5792, dtype=torch.float64, device='cuda:0')
298 tensor(14.5559, dtype=torch.float64, device='cuda:0')
299 tensor(14.5743, dtype=torch.float64, device='cuda:0')
300 tensor(14.5404, dtype=torch.float64, device='cuda:0')
301 tensor(14.5492, dtype=torch.float64, device='cuda:0')
302 tensor(14.5460, dtype=torch.float64, device='cuda:0')
303 tensor(14.5388, dtype=torch.float64, device='cuda:0')
304 tensor(14.5212, dtype=torch.float64, device='cuda:0')
305 tensor(14.5019, dtype=torch.float64, device='cuda:0')
306 tensor(14.5063, dtype=torch.float64, device='cuda:0')
307 tensor(14.4402, dtype=torch.float64, device='cuda:0')
308 tensor(14.4361, dtype=torch.float64, device='cuda:0')
309 tensor(14.4643, dtype=torch.float64, device='cuda:0')
310 tensor(14.3769, dtype=torch.float64, device='cuda:0')
311 tensor(14.

440 tensor(13.7694, dtype=torch.float64, device='cuda:0')
441 tensor(13.7768, dtype=torch.float64, device='cuda:0')
442 tensor(13.7644, dtype=torch.float64, device='cuda:0')
443 tensor(13.7558, dtype=torch.float64, device='cuda:0')
444 tensor(13.7676, dtype=torch.float64, device='cuda:0')
445 tensor(13.7662, dtype=torch.float64, device='cuda:0')
446 tensor(13.7599, dtype=torch.float64, device='cuda:0')
447 tensor(13.7274, dtype=torch.float64, device='cuda:0')
448 tensor(13.7669, dtype=torch.float64, device='cuda:0')
449 tensor(13.7348, dtype=torch.float64, device='cuda:0')
450 tensor(13.7204, dtype=torch.float64, device='cuda:0')
451 tensor(13.7536, dtype=torch.float64, device='cuda:0')
452 tensor(13.7404, dtype=torch.float64, device='cuda:0')
453 tensor(13.7429, dtype=torch.float64, device='cuda:0')
454 tensor(13.7306, dtype=torch.float64, device='cuda:0')
455 tensor(13.7288, dtype=torch.float64, device='cuda:0')
456 tensor(13.7420, dtype=torch.float64, device='cuda:0')
457 tensor(13.

...

1462 tensor(13.0174, dtype=torch.float64, device='cuda:0')
1463 tensor(13.0610, dtype=torch.float64, device='cuda:0')
1464 tensor(13.0715, dtype=torch.float64, device='cuda:0')
1465 tensor(13.0884, dtype=torch.float64, device='cuda:0')
1466 tensor(13.0871, dtype=torch.float64, device='cuda:0')
1467 tensor(13.0838, dtype=torch.float64, device='cuda:0')
1468 tensor(13.0814, dtype=torch.float64, device='cuda:0')
1469 tensor(13.0729, dtype=torch.float64, device='cuda:0')
1470 tensor(13.0738, dtype=torch.float64, device='cuda:0')
1471 tensor(13.0848, dtype=torch.float64, device='cuda:0')
1472 tensor(13.0607, dtype=torch.float64, device='cuda:0')
1473 tensor(13.0928, dtype=torch.float64, device='cuda:0')
1474 tensor(13.0960, dtype=torch.float64, device='cuda:0')
1475 tensor(13.0718, dtype=torch.float64, device='cuda:0')
1476 tensor(13.0767, dtype=torch.float64, device='cuda:0')
1477 tensor(13.0837, dtype=torch.float64, device='cuda:0')
1478 tensor(13.0774, dtype=torch.float64, device='cuda:0

1607 tensor(13.0665, dtype=torch.float64, device='cuda:0')
1608 tensor(13.0404, dtype=torch.float64, device='cuda:0')
1609 tensor(13.0617, dtype=torch.float64, device='cuda:0')
1610 tensor(13.0427, dtype=torch.float64, device='cuda:0')
1611 tensor(13.0072, dtype=torch.float64, device='cuda:0')
1612 tensor(13.0754, dtype=torch.float64, device='cuda:0')
1613 tensor(13.0591, dtype=torch.float64, device='cuda:0')
1614 tensor(13.0722, dtype=torch.float64, device='cuda:0')
1615 tensor(13.0583, dtype=torch.float64, device='cuda:0')
1616 tensor(13.0650, dtype=torch.float64, device='cuda:0')
1617 tensor(13.0750, dtype=torch.float64, device='cuda:0')
1618 tensor(13.0645, dtype=torch.float64, device='cuda:0')
1619 tensor(13.0678, dtype=torch.float64, device='cuda:0')
1620 tensor(13.0263, dtype=torch.float64, device='cuda:0')
1621 tensor(13.0849, dtype=torch.float64, device='cuda:0')
1622 tensor(13.0663, dtype=torch.float64, device='cuda:0')
1623 tensor(13.0412, dtype=torch.float64, device='cuda:0

1750 tensor(13.0515, dtype=torch.float64, device='cuda:0')
1751 tensor(13.0541, dtype=torch.float64, device='cuda:0')
1752 tensor(13.0275, dtype=torch.float64, device='cuda:0')
1753 tensor(13.0406, dtype=torch.float64, device='cuda:0')
1754 tensor(13.0511, dtype=torch.float64, device='cuda:0')
1755 tensor(13.0630, dtype=torch.float64, device='cuda:0')
1756 tensor(13.0607, dtype=torch.float64, device='cuda:0')
1757 tensor(13.0519, dtype=torch.float64, device='cuda:0')
1758 tensor(13.0576, dtype=torch.float64, device='cuda:0')
1759 tensor(13.0652, dtype=torch.float64, device='cuda:0')
1760 tensor(13.0479, dtype=torch.float64, device='cuda:0')
1761 tensor(13.0504, dtype=torch.float64, device='cuda:0')
1762 tensor(13.0472, dtype=torch.float64, device='cuda:0')
1763 tensor(13.0549, dtype=torch.float64, device='cuda:0')
1764 tensor(13.0593, dtype=torch.float64, device='cuda:0')
1765 tensor(13.0017, dtype=torch.float64, device='cuda:0')
1766 tensor(13.0731, dtype=torch.float64, device='cuda:0

1893 tensor(13.0506, dtype=torch.float64, device='cuda:0')
1894 tensor(13.0567, dtype=torch.float64, device='cuda:0')
1895 tensor(13.0444, dtype=torch.float64, device='cuda:0')
1896 tensor(13.0494, dtype=torch.float64, device='cuda:0')
1897 tensor(13.0461, dtype=torch.float64, device='cuda:0')
1898 tensor(13.0414, dtype=torch.float64, device='cuda:0')
1899 tensor(13.0486, dtype=torch.float64, device='cuda:0')
1900 tensor(13.0427, dtype=torch.float64, device='cuda:0')
1901 tensor(13.0411, dtype=torch.float64, device='cuda:0')
1902 tensor(13.0299, dtype=torch.float64, device='cuda:0')
1903 tensor(12.9770, dtype=torch.float64, device='cuda:0')
1904 tensor(13.0408, dtype=torch.float64, device='cuda:0')
1905 tensor(13.0511, dtype=torch.float64, device='cuda:0')
1906 tensor(13.0546, dtype=torch.float64, device='cuda:0')
1907 tensor(13.0536, dtype=torch.float64, device='cuda:0')
1908 tensor(13.0065, dtype=torch.float64, device='cuda:0')
1909 tensor(13.0580, dtype=torch.float64, device='cuda:0

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


In [100]:
model=train_model('shallow',1000,0.9)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(3191.9378, dtype=torch.float64, device='cuda:0')
1 tensor(1516.8671, dtype=torch.float64, device='cuda:0')
2 tensor(515.1912, dtype=torch.float64, device='cuda:0')
3 tensor(142.3038, dtype=torch.float64, device='cuda:0')
4 tensor(99.0982, dtype=torch.float64, device='cuda:0')
5 tensor(57.2768, dtype=torch.float64, device='cuda:0')
6 tensor(60.8433, dtype=torch.float64, device='cuda:0')
7 tensor(41.8576, dtype=torch.float64, device='cuda:0')
8 tensor(52.8699, dtype=torch.float64, device='cuda:0')
9 tensor(17.8679, dtype=torch.float64, device='cuda:0')
10 tensor(74.7640, dtype=torch.float64, device='cuda:0')
11 tensor(22.6426, dtype=torch.float64, device='cuda:0')
12 tensor(39.7915, dtype=torch.float64, device='cuda:0')
13 tensor(40.2151, dtype=torch.float64, device='cuda:0')
14 tensor(43.2752, dtype=torch.float64, device='cuda:0')
15 tensor(37.2796, dtype=torch.float64, device='cuda:0')
16 tensor(39.1661, dtype=torch.float64, device='cuda:0')
17 tensor(28.4127, dtype=torch.floa

148 tensor(13.1180, dtype=torch.float64, device='cuda:0')
149 tensor(14.7937, dtype=torch.float64, device='cuda:0')
150 tensor(13.8087, dtype=torch.float64, device='cuda:0')
151 tensor(15.8805, dtype=torch.float64, device='cuda:0')
152 tensor(14.0478, dtype=torch.float64, device='cuda:0')
153 tensor(14.8039, dtype=torch.float64, device='cuda:0')
154 tensor(12.7922, dtype=torch.float64, device='cuda:0')
155 tensor(13.3237, dtype=torch.float64, device='cuda:0')
156 tensor(12.4646, dtype=torch.float64, device='cuda:0')
157 tensor(12.8209, dtype=torch.float64, device='cuda:0')
158 tensor(16.1168, dtype=torch.float64, device='cuda:0')
159 tensor(12.5396, dtype=torch.float64, device='cuda:0')
160 tensor(12.4891, dtype=torch.float64, device='cuda:0')
161 tensor(14.6865, dtype=torch.float64, device='cuda:0')
162 tensor(14.0233, dtype=torch.float64, device='cuda:0')
163 tensor(13.2786, dtype=torch.float64, device='cuda:0')
164 tensor(14.2470, dtype=torch.float64, device='cuda:0')
165 tensor(14.

294 tensor(9.5407, dtype=torch.float64, device='cuda:0')
295 tensor(11.3425, dtype=torch.float64, device='cuda:0')
296 tensor(8.9319, dtype=torch.float64, device='cuda:0')
297 tensor(8.0804, dtype=torch.float64, device='cuda:0')
298 tensor(7.9449, dtype=torch.float64, device='cuda:0')
299 tensor(8.0615, dtype=torch.float64, device='cuda:0')
300 tensor(9.3972, dtype=torch.float64, device='cuda:0')
301 tensor(7.4290, dtype=torch.float64, device='cuda:0')
302 tensor(8.8026, dtype=torch.float64, device='cuda:0')
303 tensor(8.4134, dtype=torch.float64, device='cuda:0')
304 tensor(8.6619, dtype=torch.float64, device='cuda:0')
305 tensor(8.6758, dtype=torch.float64, device='cuda:0')
306 tensor(8.4612, dtype=torch.float64, device='cuda:0')
307 tensor(9.3804, dtype=torch.float64, device='cuda:0')
308 tensor(9.7025, dtype=torch.float64, device='cuda:0')
309 tensor(7.8288, dtype=torch.float64, device='cuda:0')
310 tensor(8.7548, dtype=torch.float64, device='cuda:0')
311 tensor(9.0842, dtype=torch

438 tensor(7.8889, dtype=torch.float64, device='cuda:0')
439 tensor(7.6434, dtype=torch.float64, device='cuda:0')
440 tensor(7.9423, dtype=torch.float64, device='cuda:0')
441 tensor(7.1863, dtype=torch.float64, device='cuda:0')
442 tensor(8.0101, dtype=torch.float64, device='cuda:0')
443 tensor(6.4922, dtype=torch.float64, device='cuda:0')
444 tensor(6.5512, dtype=torch.float64, device='cuda:0')
445 tensor(7.3320, dtype=torch.float64, device='cuda:0')
446 tensor(7.2408, dtype=torch.float64, device='cuda:0')
447 tensor(6.9719, dtype=torch.float64, device='cuda:0')
448 tensor(7.6704, dtype=torch.float64, device='cuda:0')
449 tensor(7.5757, dtype=torch.float64, device='cuda:0')
450 tensor(7.1752, dtype=torch.float64, device='cuda:0')
451 tensor(7.2460, dtype=torch.float64, device='cuda:0')
452 tensor(6.7107, dtype=torch.float64, device='cuda:0')
453 tensor(6.9678, dtype=torch.float64, device='cuda:0')
454 tensor(7.1581, dtype=torch.float64, device='cuda:0')
455 tensor(6.9905, dtype=torch.

585 tensor(6.6413, dtype=torch.float64, device='cuda:0')
586 tensor(6.6755, dtype=torch.float64, device='cuda:0')
587 tensor(6.7069, dtype=torch.float64, device='cuda:0')
588 tensor(6.7463, dtype=torch.float64, device='cuda:0')
589 tensor(6.7894, dtype=torch.float64, device='cuda:0')
590 tensor(7.2643, dtype=torch.float64, device='cuda:0')
591 tensor(7.2166, dtype=torch.float64, device='cuda:0')
592 tensor(6.4911, dtype=torch.float64, device='cuda:0')
593 tensor(6.6364, dtype=torch.float64, device='cuda:0')
594 tensor(6.8250, dtype=torch.float64, device='cuda:0')
595 tensor(6.8826, dtype=torch.float64, device='cuda:0')
596 tensor(6.5917, dtype=torch.float64, device='cuda:0')
597 tensor(6.6489, dtype=torch.float64, device='cuda:0')
598 tensor(7.2590, dtype=torch.float64, device='cuda:0')
599 tensor(7.0814, dtype=torch.float64, device='cuda:0')
600 tensor(6.6794, dtype=torch.float64, device='cuda:0')
601 tensor(6.6359, dtype=torch.float64, device='cuda:0')
602 tensor(6.5384, dtype=torch.

732 tensor(7.0689, dtype=torch.float64, device='cuda:0')
733 tensor(6.8904, dtype=torch.float64, device='cuda:0')
734 tensor(6.4347, dtype=torch.float64, device='cuda:0')
735 tensor(6.6113, dtype=torch.float64, device='cuda:0')
736 tensor(7.0961, dtype=torch.float64, device='cuda:0')
737 tensor(6.3789, dtype=torch.float64, device='cuda:0')
738 tensor(6.6184, dtype=torch.float64, device='cuda:0')
739 tensor(6.4001, dtype=torch.float64, device='cuda:0')
740 tensor(6.8373, dtype=torch.float64, device='cuda:0')
741 tensor(6.4930, dtype=torch.float64, device='cuda:0')
742 tensor(6.5018, dtype=torch.float64, device='cuda:0')
743 tensor(6.8838, dtype=torch.float64, device='cuda:0')
744 tensor(6.7854, dtype=torch.float64, device='cuda:0')
745 tensor(6.7965, dtype=torch.float64, device='cuda:0')
746 tensor(6.5936, dtype=torch.float64, device='cuda:0')
747 tensor(6.7229, dtype=torch.float64, device='cuda:0')
748 tensor(6.7261, dtype=torch.float64, device='cuda:0')
749 tensor(6.4882, dtype=torch.

881 tensor(6.8291, dtype=torch.float64, device='cuda:0')
882 tensor(6.1376, dtype=torch.float64, device='cuda:0')
883 tensor(6.2404, dtype=torch.float64, device='cuda:0')
884 tensor(5.9159, dtype=torch.float64, device='cuda:0')
885 tensor(6.3209, dtype=torch.float64, device='cuda:0')
886 tensor(6.2561, dtype=torch.float64, device='cuda:0')
887 tensor(6.3507, dtype=torch.float64, device='cuda:0')
888 tensor(6.3021, dtype=torch.float64, device='cuda:0')
889 tensor(6.5655, dtype=torch.float64, device='cuda:0')
890 tensor(6.4454, dtype=torch.float64, device='cuda:0')
891 tensor(6.6251, dtype=torch.float64, device='cuda:0')
892 tensor(6.0450, dtype=torch.float64, device='cuda:0')
893 tensor(6.1683, dtype=torch.float64, device='cuda:0')
894 tensor(6.3144, dtype=torch.float64, device='cuda:0')
895 tensor(6.6328, dtype=torch.float64, device='cuda:0')
896 tensor(6.2160, dtype=torch.float64, device='cuda:0')
897 tensor(6.3989, dtype=torch.float64, device='cuda:0')
898 tensor(6.3649, dtype=torch.

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


In [126]:
def prediction(data,model):
    assert isinstance(data,np.ndarray)
    data=Variable(torch.from_numpy(data),requires_grad = False)
    #assert data.shape[1]==19
    #normalize data according to original dataset to fit the model
    data=(data- dataMin) / (dataMax - dataMin)
    newmodel=model.cpu()
    return newmodel(data).item()
    
    

In [137]:
testdata=np.array(data.iloc[2039])
#print(testdata)
result=prediction(testdata,model)
print(result)

77.5349320090655


In [ ]:
#dataset has't been divided into training set, test set, validation set
#